# This is my first attempt at cleaning and feature engineering for the Kaggle submission

### Instead of having a separate notebook for my train.csv and test.csv, I just cleaned train.csv first, then changed the filepaths and reran the notebook on the test.csv data. Lazy, yes. Best practice, probably not.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from local_EDA import heat # This is my local helper functions .py file

Standard imports above, reading in the csvs below

In [2]:
df = pd.read_csv('datasets/test.csv')

Standard opening moves for EDA: check `.head`, `.info`, etc. looking for `NaN`s, mismatched `dtype`s, unrealistic values, etc.

In [3]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

In [5]:
df['Misc Feature'].value_counts(dropna=False)

NaN     837
Shed     39
Gar2      1
Othr      1
Name: Misc Feature, dtype: int64

In [8]:
df['Lot Frontage'].describe()

count    718.000000
mean      69.545961
std       23.533945
min       21.000000
25%       59.000000
50%       68.000000
75%       80.000000
max      182.000000
Name: Lot Frontage, dtype: float64

In [9]:
df['Lot Area'].describe()

count       878.000000
mean      10307.033030
std       10002.674602
min        1477.000000
25%        7297.250000
50%        9446.000000
75%       11589.000000
max      215245.000000
Name: Lot Area, dtype: float64

In [10]:
df['Year Built'].describe()

count     878.000000
mean     1970.492027
std        30.395895
min      1880.000000
25%      1954.000000
50%      1972.000000
75%      1999.750000
max      2010.000000
Name: Year Built, dtype: float64

In [11]:
df['Year Remod/Add'].isna().sum()

0

In [12]:
df['Year Remod/Add'].describe()

count     878.000000
mean     1984.417995
std        20.450725
min      1950.000000
25%      1967.000000
50%      1992.000000
75%      2003.000000
max      2010.000000
Name: Year Remod/Add, dtype: float64

In [13]:
df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

Here I dropped a bunch of columns that I thought (after reading the <a href='http://jse.amstat.org/v19n3/decock/DataDocumentation.txt'>data dictionary</a>) would do more harm than good in prediction. Some of these would be collinear, some wouldn't fit well into my vision for how I wanted to clean my data, some were redundant.

In [14]:
df.drop(columns=['PID', 'MS SubClass', 'Utilities', 'Condition 2', 'Year Remod/Add', 'Exterior 2nd', 
                'Mas Vnr Area', 'Bsmt Exposure', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 
                 'Bsmt Unf SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Garage Yr Blt',
                'Garage Cars', 'Garage Qual', 'Garage Cond', 'Pool QC', 'Misc Val', 'Sale Type'],
       inplace=True)

In [15]:
df.head()

,Id,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Lot Config,Land Slope,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Fence,Misc Feature,Mo Sold,Yr Sold
0,2658,RM,69.0,9142,Pave,Grvl,Reg,Lvl,Inside,Gtl,...,0,60,112,0,0,0,NaN,NaN,4,2006
1,2718,RL,NaN,9662,Pave,NaN,IR1,Lvl,Inside,Gtl,...,170,0,0,0,0,0,NaN,NaN,8,2006
2,2414,RL,58.0,17104,Pave,NaN,IR1,Lvl,Inside,Gtl,...,100,24,0,0,0,0,NaN,NaN,9,2006
3,1989,RM,60.0,8520,Pave,NaN,Reg,Lvl,Inside,Gtl,...,0,0,184,0,0,0,NaN,NaN,7,2007
4,625,RL,NaN,9500,Pave,NaN,IR1,Lvl,Inside,Gtl,...,0,76,0,0,185,0,NaN,NaN,7,2009


I decided that the easiest way to clean all of these nasty categorical features, since so many of them were unbalanced to begin with, was to simply binarize most of them.

In [16]:
# Separating into Residential zoning or not
df['MS Zoning'] = df['MS Zoning'].map(lambda x: 1 if x[0] == 'R' else 0)
df['MS Zoning'].value_counts()

1    833
0     45
Name: MS Zoning, dtype: int64

In [17]:
df['Lot Frontage'].isna().sum()

160

 I'm assuming the NaN values mean 0, and separating into "within IQR of frontage or not"

In [ ]:
df['Lot Frontage'] = df['Lot Frontage'].map(lambda x: 1 if 58 <= x <= 80 else 0)
df['Lot Frontage'].value_counts()

In [19]:
df['Lot Area'].describe()

count       878.000000
mean      10307.033030
std       10002.674602
min        1477.000000
25%        7297.250000
50%        9446.000000
75%       11589.000000
max      215245.000000
Name: Lot Area, dtype: float64

In [20]:
df['Lot Area'].value_counts(dropna=False).sort_index(ascending=False).head(10)

215245    1
164660    1
56600     1
45600     1
36500     1
35760     1
35133     1
33983     1
31220     1
29959     1
Name: Lot Area, dtype: int64

This one was already binary, just converting to actual 1/0

In [21]:
df.Street = df.Street.map(lambda x: 1 if x == 'Pave' else 0)

Whether or not a house has alley access

In [22]:
df.Alley = df.Alley.map(lambda x: 1 if x in ['Grvl', 'Pave'] else 0)

Whether or not the lot is regularly shaped

In [23]:
df['Lot Shape'] = df['Lot Shape'].map(lambda x: 1 if x == 'Reg' else 0)

Whether or not the lot is flat

In [24]:
df['Land Contour'] = df['Land Contour'].map(lambda x: 1 if x == 'Lvl' else 0)

I got sick of writing out unique lambdas for every column

In [25]:
# Laziness is my passion

# This lets me input a column to map, an operator (is, is not, in, greater than, etc.), 
# and a condition to compare to, and returns a mapped (binarized) version of the input column.
# I could have added more operators, but I only needed the ones you see below
def binarizer(col, operator, condition):
    if operator == 'is':
        return df[col].map(lambda x: 1 if x == condition else 0)
    elif operator == 'is not':
        return df[col].map(lambda x: 1 if x != condition else 0)
    elif operator == 'lt':
        return df[col].map(lambda x: 1 if x < condition else 0)
    elif operator == 'in':
        return df[col].map(lambda x: 1 if x in condition else 0)
    elif operator == 'gte':
        return df[col].map(lambda x: 1 if x >= condition else 0)
    

Wherther or not the lot is 'normal' (specifically an inside lot, not corner lot, cul-de-sac, other weird lot type)

In [26]:
df['Lot Config'] = binarizer('Lot Config', 'is', 'Inside')

In [27]:
df['Condition 1'].value_counts()

Norm      755
Feedr      55
Artery     22
RRAn       14
PosN       11
PosA        8
RRAe        7
RRNe        3
RRNn        3
Name: Condition 1, dtype: int64

I made this one essentially `is_near_railroad`, as most of the values here are 'no condition' anyway, and I predicted that proximity to a railroad would have the most impact on house value

In [28]:
df['Condition 1'] = binarizer('Condition 1', 'in', ['RRAn', 'RRAe', 'RRNn', 'RRNe'])

Single family house or not

In [29]:
df['Bldg Type'] = binarizer('Bldg Type', 'is', '1Fam')

Single story house or not

In [30]:
df['House Style'] = binarizer('House Style', 'is', '1Story')

House built before 1970 or not

In [31]:
df['Year Built'] = binarizer('Year Built', 'lt', 1970)

Gable style roof or not

In [32]:
df['Roof Style'] = binarizer('Roof Style', 'is', 'Gable')

Standard composite shingled roof or not

In [33]:
df['Roof Matl'] = binarizer('Roof Matl', 'is', 'CompShg')

Standard vinyl siding or not

In [34]:
df['Exterior 1st'] = binarizer('Exterior 1st', 'is', 'VinylSd')

Has masonry veneer or not

In [35]:
df['Mas Vnr Type'] = binarizer('Mas Vnr Type', 'is not', 'None')

Exterior quality above average or not

In [36]:
df['Exter Qual'] = binarizer('Exter Qual', 'in', ['Gd', 'Ex'])

In [37]:
gdex = ['Gd', 'Ex']

Current exterior condition above average or not

In [38]:
df['Exter Cond'] = binarizer('Exter Cond', 'in', gdex)

Basement ceiling height above average or not

In [39]:
df['Bsmt Qual'] = binarizer('Bsmt Qual', 'in', gdex)

Basement condition above average or not

In [40]:
df['Bsmt Cond'] = binarizer('Bsmt Cond', 'in', gdex)

Is the basement finished or not

In [42]:
df['BsmtFin Type 1'] = binarizer('BsmtFin Type 1', 'in', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ'])

Has forced air furnace or not

In [43]:
df.Heating = binarizer('Heating', 'is', 'GasA')

Heating quality above average or not

In [44]:
df['Heating QC'] = binarizer('Heating QC', 'in', gdex)

Has central air or not

In [45]:
df['Central Air'] = binarizer('Central Air', 'is', 'Y')

Has standard circuit breaker or not

In [46]:
df.Electrical = binarizer('Electrical', 'is', 'SBrkr')

Kitchen quality above average or not

In [47]:
df['Kitchen Qual'] = binarizer('Kitchen Qual', 'in', gdex)

Is the house fully functional or not

In [48]:
df.Functional = binarizer('Functional', 'is', 'Typ')

Has a fireplace or not

In [49]:
df.Fireplaces = binarizer('Fireplaces', 'gte', 1)

fireplace quality above average or not

In [50]:
df['Fireplace Qu'] = binarizer('Fireplace Qu', 'in', gdex)

Assuming `NaN`s for `Garage Type` indicate no garage, filling `NaN`s with 0

In [51]:
df['Garage Type'].fillna(0, inplace=True)
df['Garage Type'].value_counts(dropna=False)

Attchd     518
Detchd     246
BuiltIn     53
0           44
Basment      9
CarPort      4
2Types       4
Name: Garage Type, dtype: int64

Has a garage or not

In [52]:
# has a garage or not
df['Garage Type'] = binarizer('Garage Type', 'is not', 0)

Is the garage finished or not

In [54]:
df['Garage Finish'] = df['Garage Finish'].map(lambda x: 1 if x in ['RFn', 'Fin'] else 0)

Is the driveway paved or not

In [55]:
df['Paved Drive'] = binarizer('Paved Drive', 'in', ['Y', 'P'])

For each of the deck/porch columns and the pool column, does the house have one or not

In [57]:
cols = ['Wood Deck SF', 'Open Porch SF',
       'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area']
for col in cols:
    df[col] = binarizer(col, 'gte', 1)

In [59]:
df.Fence = binarizer('Fence', 'in', ['MnPrv', 'GdPrv', 'GdWo', 'MnWw'])

Does the house have a miscellanious feature or not

In [61]:
df['Misc Feature'] = binarizer('Misc Feature', 'in', ['Shed', 'Gar2', 'Othr', 'TenC', 'Elev'])

Combining features into aggregates that make more sense:

I.e. making totals out of above/below grade room/sqft counts

In [62]:
df['total_sqft'] = df['Total Bsmt SF'] + df['Gr Liv Area']

In [63]:
df['full_baths'] = df['Bsmt Full Bath'] + df['Full Bath']

In [64]:
df['half_baths'] = df['Bsmt Half Bath'] + df['Half Bath']

In [65]:
df['total_rooms'] = df.full_baths + df.half_baths + df['Kitchen AbvGr'] + df['Bedroom AbvGr']

Then dropping the columns we no longer need

In [66]:
df.drop(columns=['Total Bsmt SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Full Bath', 'Bsmt Half Bath', 
                'Half Bath', 'Kitchen AbvGr', 'Bedroom AbvGr'], inplace=True)

In [69]:
df.drop(columns=['TotRms AbvGrd'], inplace=True)

And renaming the columns into something more friendly to my lazy typing fingers:

In [71]:
df.rename(columns = {'MS Zoning': 'zoning', 
                     'Lot Frontage': 'normal_frontage', 
                     'Lot Area': 'lot_area',
                     'Street': 'paved_access',
                     'Alley': 'alley', 
                     'Lot Shape': 'reg_shape', 
                     'Land Contour': 'is_flat', 
                     'Lot Config': 'inside_lot', 
                     'Land Slope': 'slope', 
                     'Neighborhood': 'neighborhood', 
                     'Condition 1': 'near_rr', 
                     'Bldg Type': 'single_fam', 
                     'House Style': 'one_story', 
                     'Overall Qual': 'house_qual', 
                     'Overall Cond': 'house_cond', 
                     'Year Built': 'built_before_1970', 
                     'Roof Style': 'roof_gable', 
                     'Roof Matl': 'roof_std_shingle', 
                     'Exterior 1st': 'vinyl_siding', 
                     'Mas Vnr Type': 'has_masonry', 
                     'Exter Qual': 'ext_qual_above_avg', 
                     'Exter Cond': 'ext_cond_above_avg',
                     'Foundation': 'foundation', 
                     'Bsmt Qual': 'b_height_above_avg', 
                     'Bsmt Cond': 'b_cond_above_avg', 
                     'BsmtFin Type 1': 'basement_finished', 
                     'Heating': 'forced_air', 
                     'Heating QC': 'heating_above_avg', 
                     'Central Air': 'central_air', 
                     'Electrical': 'std_circuit_breaker', 
                     'Kitchen Qual': 'kitchen_above_avg', 
                     'Functional': 'fully_functional', 
                     'Fireplaces': 'has_fireplace', 
                     'Fireplace Qu': 'fireplace_qual', 
                     'Garage Type': 'has_garage', 
                     'Garage Finish': 'garage_finished', 
                     'Garage Area': 'garage_sqft', 
                     'Paved Drive': 'driveway_paved', 
                     'Wood Deck SF': 'has_wood_deck', 
                     'Open Porch SF': 'has_open_porch', 
                     'Enclosed Porch': 'has_encl_porch', 
                     '3Ssn Porch': 'has_3ssn_porch', 
                     'Screen Porch': 'has_screen_porch', 
                     'Pool Area': 'has_pool', 
                     'Fence': 'has_fence', 
                     'Misc Feature': 'has_misc', 
                     'Mo Sold': 'month_sold', 
                     'Yr Sold': 'year_sold'
                    }, inplace=True)

Double-checking I didn't miss anything

In [72]:
df.columns

Index(['Id', 'zoning', 'normal_frontage', 'lot_area', 'paved_access', 'alley',
       'reg_shape', 'is_flat', 'inside_lot', 'slope', 'neighborhood',
       'near_rr', 'single_fam', 'one_story', 'house_qual', 'house_cond',
       'built_before_1970', 'roof_gable', 'roof_std_shingle', 'vinyl_siding',
       'has_masonry', 'ext_qual_above_avg', 'ext_cond_above_avg', 'foundation',
       'b_height_above_avg', 'b_cond_above_avg', 'basement_finished',
       'forced_air', 'heating_above_avg', 'central_air', 'std_circuit_breaker',
       'kitchen_above_avg', 'fully_functional', 'has_fireplace',
       'fireplace_qual', 'has_garage', 'garage_finished', 'garage_sqft',
       'driveway_paved', 'has_wood_deck', 'has_open_porch', 'has_encl_porch',
       'has_3ssn_porch', 'has_screen_porch', 'has_pool', 'has_fence',
       'has_misc', 'month_sold', 'year_sold', 'total_sqft', 'full_baths',
       'half_baths', 'total_rooms'],
      dtype='object')

In [73]:
df.isna().sum()

Id                     0
zoning                 0
normal_frontage        0
lot_area               0
paved_access           0
alley                  0
reg_shape              0
is_flat                0
inside_lot             0
slope                  0
neighborhood           0
near_rr                0
single_fam             0
one_story              0
house_qual             0
house_cond             0
built_before_1970      0
roof_gable             0
roof_std_shingle       0
vinyl_siding           0
has_masonry            0
ext_qual_above_avg     0
ext_cond_above_avg     0
foundation             0
b_height_above_avg     0
b_cond_above_avg       0
basement_finished      0
forced_air             0
heating_above_avg      0
central_air            0
std_circuit_breaker    0
kitchen_above_avg      0
fully_functional       0
has_fireplace          0
fireplace_qual         0
has_garage             0
garage_finished        0
garage_sqft            0
driveway_paved         0
has_wood_deck          0


In [74]:
df.garage_sqft.fillna(df.garage_sqft.mean(), inplace=True)

In [75]:
for col in ['total_sqft', 'full_baths', 'half_baths', 'total_rooms']:
    df[col].fillna(df[col].mean(), inplace=True)

In [76]:
df.isna().sum()

Id                     0
zoning                 0
normal_frontage        0
lot_area               0
paved_access           0
alley                  0
reg_shape              0
is_flat                0
inside_lot             0
slope                  0
neighborhood           0
near_rr                0
single_fam             0
one_story              0
house_qual             0
house_cond             0
built_before_1970      0
roof_gable             0
roof_std_shingle       0
vinyl_siding           0
has_masonry            0
ext_qual_above_avg     0
ext_cond_above_avg     0
foundation             0
b_height_above_avg     0
b_cond_above_avg       0
basement_finished      0
forced_air             0
heating_above_avg      0
central_air            0
std_circuit_breaker    0
kitchen_above_avg      0
fully_functional       0
has_fireplace          0
fireplace_qual         0
has_garage             0
garage_finished        0
garage_sqft            0
driveway_paved         0
has_wood_deck          0


And dummifying our remaining, unbinarizable columns

In [77]:
df = pd.get_dummies(df, columns=['slope', 'neighborhood', 'foundation', 'month_sold', 'year_sold'], 
                   drop_first=True)

This puts the target column at the end of the DataFrame

In [78]:
# tgt = df['SalePrice']
# df = df.drop(columns=['SalePrice'])
# df.insert(loc=len(df.columns), column='SalePrice', value=tgt)

KeyError: 'SalePrice'

And finally, saving it all to a .csv

In [81]:
# df.to_csv('./datasets/test_cleaned.csv', index=False)